In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-sms-detection/spam.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [3]:
df = pd.read_csv('/kaggle/input/spam-sms-detection/spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [5]:
import nltk
df.columns = ['label', 'Content']


en_stopwords = nltk.corpus.stopwords.words("english")


ps = nltk.PorterStemmer()

In [6]:
df["Content_len"] = df["Content"].apply(lambda x : len(x)- x.count(" "))

In [7]:
import string

def countPunctuation(email) :
    binary_array = [1 for ch in email if ch in string.punctuation]
    total_punctuation = sum(binary_array)
    total  = len(email) - email.count(" ")
    return round(total_punctuation/(total),4) * 100


df["punctuation_taux"] = df["Content"].apply(lambda x : countPunctuation(x))

In [8]:
X = df.Content
Y = df.label
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 1000


max_len = 150


tok = Tokenizer(num_words=max_words)


tok.fit_on_texts(X_train)


sequences = tok.texts_to_sequences(X_train)

sequences_matrix = pad_sequences(sequences, maxlen=max_len)

In [10]:
sequences_matrix

array([[  0,   0,   0, ...,  25, 782, 256],
       [  0,   0,   0, ..., 225,  21,  99],
       [  0,   0,   0, ...,   4, 668, 306],
       ...,
       [  0,   0,   0, ..., 705, 574, 885],
       [  0,   0,   0, ..., 130,   3,  29],
       [  0,   0,   0, ...,   0,   0, 954]], dtype=int32)

In [11]:
X_train.shape

(4736,)

In [12]:
sequences_matrix.shape

(4736, 150)

In [13]:
Y_train.shape

(4736, 1)

In [14]:
sequences_matrix

array([[  0,   0,   0, ...,  25, 782, 256],
       [  0,   0,   0, ..., 225,  21,  99],
       [  0,   0,   0, ...,   4, 668, 306],
       ...,
       [  0,   0,   0, ..., 705, 574, 885],
       [  0,   0,   0, ..., 130,   3,  29],
       [  0,   0,   0, ...,   0,   0, 954]], dtype=int32)

In [15]:
def RNN():
 
    inputs = Input(name='inputs', shape=[max_len])
    
  
    layer = Embedding(max_words, 50, input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1, name='out_layer')(layer)

    model = Model(inputs=inputs, outputs=layer)
    
    return model

In [16]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)             │ (None, 150)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 150, 50)        │        50,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        29,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC1 (Dense)                     │ (None, 256)            │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ out_layer (Dense)               │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 96,337 (376.32 KB)

 Trainable params: 96,337 (376.32 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, mode='min')
csv_logger = CSVLogger('training_log.csv', append=True)

#
history = model.fit(
    sequences_matrix, 
    Y_train,          
    batch_size=128,   
    epochs=2,          
    validation_split=0.2,  
    callbacks=[early_stopping, csv_logger]  
)

Epoch 1/2
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 183ms/step - accuracy: 0.8700 - loss: 0.5424 - val_accuracy: 0.9757 - val_loss: 0.1455
Epoch 2/2
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 178ms/step - accuracy: 0.9721 - loss: 0.1499 - val_accuracy: 0.9884 - val_loss: 0.1050


In [18]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [19]:
score = model.evaluate(test_sequences_matrix,Y_test)

27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9842 - loss: 0.0625


In [20]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(score[0],score[1]))

Test set
  Loss: 0.103
  Accuracy: 0.981
